In [1]:
!pip install transformers accelerate bitsandbytes huggingface_hub -U --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 93.7 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 26.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 66.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("")


In [3]:
from huggingface_hub import login
login(secret_value_0)


In [5]:
!pip install sqlglot --quiet

In [8]:
import sqlglot
from sqlglot import exp

def extract_query_features(sql):
    """
    Extract tables, columns, join types & conditions, and aggregation functions from a SQL query.
    """
    try:
        expression = sqlglot.parse_one(sql)
    except Exception as e:
        return {"error": str(e)}

    tables = set()
    columns = set()
    joins = []
    aggregations = set()

    # --- Tables ---
    for table in expression.find_all(exp.Table):
        tables.add(table.name)

    # --- Columns ---
    for col in expression.find_all(exp.Column):
        columns.add(col.name)

    # --- Joins and Join Conditions ---
    for join in expression.find_all(exp.Join):
        join_type = join.args.get("kind", "INNER").upper()  # Default to INNER
        condition = None
        # The join expression stores its condition in 'on'
        if "on" in join.args and join.args["on"]:
            condition = join.args["on"].sql()
        joins.append({"type": join_type, "condition": condition})

    # --- Aggregation Functions ---
    for func in expression.find_all(exp.Func):
        func_name = func.sql_name().upper()
        if func_name in {"SUM", "COUNT", "AVG", "MIN", "MAX"}:
            aggregations.add(func_name)

    return {
        "tables": list(tables),
        "columns": list(columns),
        "joins": joins,
        "aggregations": list(aggregations)
    }

# --- Example Usage ---
queries = [
    "SELECT c.name, SUM(o.amount) FROM customers c JOIN orders o ON c.id=o.customer_id GROUP BY c.name;",
    "SELECT COUNT(*) FROM orders WHERE customer_id = 42;",
    "UPDATE inventory SET stock=stock-1 WHERE product_id=101;"
]

for q in queries:
    features = extract_query_features(q)
    print(f"\nQuery: {q}")
    for k, v in features.items():
        print(f"{k.capitalize()}: {v}")



Query: SELECT c.name, SUM(o.amount) FROM customers c JOIN orders o ON c.id=o.customer_id GROUP BY c.name;
Tables: ['orders', 'customers']
Columns: ['customer_id', 'name', 'id', 'amount']
Joins: [{'type': 'INNER', 'condition': 'c.id = o.customer_id'}]
Aggregations: ['SUM']

Query: SELECT COUNT(*) FROM orders WHERE customer_id = 42;
Tables: ['orders']
Columns: ['customer_id']
Joins: []
Aggregations: ['COUNT']

Query: UPDATE inventory SET stock=stock-1 WHERE product_id=101;
Tables: ['inventory']
Columns: ['stock', 'product_id']
Joins: []
Aggregations: []


In [4]:
# Example collected stats (as if from Performance Schema)
mysql_stats = [
    {
        "sql_text": "SELECT * FROM orders WHERE customer_id = 42;",
        "exec_time_ms": 180,
        "rows_sent": 120,
        "rows_examined": 10000,
        "join_type": "SIMPLE",
        "index_used": False
    },
    {
        "sql_text": "SELECT * FROM orders WHERE customer_id = 99;",
        "exec_time_ms": 200,
        "rows_sent": 100,
        "rows_examined": 9500,
        "join_type": "SIMPLE",
        "index_used": False
    },
    {
        "sql_text": "SELECT c.name, SUM(o.amount) FROM customers c JOIN orders o ON c.id=o.customer_id GROUP BY c.name;",
        "exec_time_ms": 7200,
        "rows_sent": 450,
        "rows_examined": 500000,
        "join_type": "HASH JOIN",
        "index_used": True
    }
]


In [5]:
import re
from collections import defaultdict

def normalize_query(sql):
    # Replace literals for grouping
    sql = re.sub(r"'[^']*'", "'?'", sql)
    sql = re.sub(r'"[^"]*"', '"?"', sql)
    sql = re.sub(r'\b\d+(\.\d+)?\b', '?', sql)
    return re.sub(r'\s+', ' ', sql).strip().lower()

# Group stats by normalized template
groups = defaultdict(list)
for entry in mysql_stats:
    template = normalize_query(entry["sql_text"])
    groups[template].append(entry)

# Aggregate metrics for each group
aggregated = []
for template, entries in groups.items():
    avg_exec_time = sum(e["exec_time_ms"] for e in entries) / len(entries)
    avg_rows_examined = sum(e["rows_examined"] for e in entries) / len(entries)
    index_usage_rate = sum(1 for e in entries if e["index_used"]) / len(entries)
    aggregated.append({
        "template": template,
        "frequency": len(entries),
        "avg_exec_time_ms": avg_exec_time,
        "avg_rows_examined": avg_rows_examined,
        "index_usage_rate": index_usage_rate,
        "join_types": list({e["join_type"] for e in entries})
    })

# Display results
for agg in aggregated:
    print(f"\nTemplate: {agg['template']}")
    print(f"Frequency: {agg['frequency']}")
    print(f"Average Exec Time (ms): {agg['avg_exec_time_ms']:.2f}")
    print(f"Average Rows Examined: {agg['avg_rows_examined']:.0f}")
    print(f"Index Usage Rate: {agg['index_usage_rate']*100:.0f}%")
    print(f"Join Types: {agg['join_types']}")



Template: select * from orders where customer_id = ?;
Frequency: 2
Average Exec Time (ms): 190.00
Average Rows Examined: 9750
Index Usage Rate: 0%
Join Types: ['SIMPLE']

Template: select c.name, sum(o.amount) from customers c join orders o on c.id=o.customer_id group by c.name;
Frequency: 1
Average Exec Time (ms): 7200.00
Average Rows Examined: 500000
Index Usage Rate: 100%
Join Types: ['HASH JOIN']


In [6]:
import pandas as pd

# --- Aggregated queries from earlier ---
aggregated_queries = [
    {
        "template": "select * from orders where customer_id = ?;",
        "frequency": 2,
        "avg_exec_time_ms": 190.0,
        "avg_rows_examined": 9750,
        "index_usage_rate": 0.0,
        "join_types": ["SIMPLE"]
    },
    {
        "template": "select c.name, sum(o.amount) from customers c join orders o on c.id=o.customer_id group by c.name;",
        "frequency": 1,
        "avg_exec_time_ms": 7200.0,
        "avg_rows_examined": 500000,
        "index_usage_rate": 1.0,
        "join_types": ["HASH JOIN"]
    },
    {
        "template": "update inventory set stock=stock-? where product_id=?;",
        "frequency": 2,
        "avg_exec_time_ms": 80.0,
        "avg_rows_examined": 50,
        "index_usage_rate": 0.5,
        "join_types": ["SIMPLE"]
    }
]

# --- Index metadata ---
indexes_df = pd.DataFrame([
    {"TABLE_NAME": "orders",    "INDEX_NAME": "PRIMARY",       "COLUMN_NAME": "id",           "NON_UNIQUE": 0},
    {"TABLE_NAME": "orders",    "INDEX_NAME": "idx_customer",  "COLUMN_NAME": "customer_id",  "NON_UNIQUE": 1},
    {"TABLE_NAME": "customers", "INDEX_NAME": "PRIMARY",       "COLUMN_NAME": "id",           "NON_UNIQUE": 0},
    {"TABLE_NAME": "inventory", "INDEX_NAME": "PRIMARY",       "COLUMN_NAME": "product_id",   "NON_UNIQUE": 0}
])

# --- Table sizes and row counts ---
table_sizes_df = pd.DataFrame([
    {"TABLE_NAME": "orders",    "row_count": 500000, "size_mb": 820.5},
    {"TABLE_NAME": "customers", "row_count": 50000,  "size_mb": 45.3},
    {"TABLE_NAME": "inventory", "row_count": 10000,  "size_mb": 12.7}
])

# --- Foreign keys and constraints ---
constraints_df = pd.DataFrame([
    {"TABLE_NAME": "orders",    "CONSTRAINT_NAME": "fk_orders_customer",
     "CONSTRAINT_TYPE": "FOREIGN KEY", "COLUMN_NAME": "customer_id",
     "REFERENCED_TABLE_NAME": "customers", "REFERENCED_COLUMN_NAME": "id"},
    {"TABLE_NAME": "orders",    "CONSTRAINT_NAME": "PRIMARY",
     "CONSTRAINT_TYPE": "PRIMARY KEY", "COLUMN_NAME": "id",
     "REFERENCED_TABLE_NAME": None,    "REFERENCED_COLUMN_NAME": None},
    {"TABLE_NAME": "customers", "CONSTRAINT_NAME": "PRIMARY",
     "CONSTRAINT_TYPE": "PRIMARY KEY", "COLUMN_NAME": "id",
     "REFERENCED_TABLE_NAME": None,    "REFERENCED_COLUMN_NAME": None},
    {"TABLE_NAME": "inventory", "CONSTRAINT_NAME": "PRIMARY",
     "CONSTRAINT_TYPE": "PRIMARY KEY", "COLUMN_NAME": "product_id",
     "REFERENCED_TABLE_NAME": None,    "REFERENCED_COLUMN_NAME": None}
])

# --- Partitioning info (assume none used) ---
partitions_df = pd.DataFrame(columns=[
    "TABLE_NAME", "PARTITION_NAME", "PARTITION_METHOD", "PARTITION_EXPRESSION"
])


In [7]:
# --- Assume these DataFrames already exist ---
# indexes_df: TABLE_NAME, INDEX_NAME, COLUMN_NAME, NON_UNIQUE
# table_sizes_df: TABLE_NAME, row_count, size_mb
# constraints_df: TABLE_NAME, CONSTRAINT_NAME, CONSTRAINT_TYPE, COLUMN_NAME, REFERENCED_TABLE_NAME, REFERENCED_COLUMN_NAME
# partitions_df: TABLE_NAME, PARTITION_NAME, PARTITION_METHOD, PARTITION_EXPRESSION
# aggregated_queries: output from grouping & aggregating runtime stats earlier

# Build a quick lookup for table sizes and indexes
table_size_map = table_sizes_df.set_index('TABLE_NAME')[['row_count', 'size_mb']].to_dict('index')
index_map = indexes_df.groupby('TABLE_NAME')['COLUMN_NAME'].apply(list).to_dict()

def enrich_query_context(aggregated_queries):
    enriched = []
    for entry in aggregated_queries:
        tables = re.findall(r'from\s+(\w+)', entry['template']) + re.findall(r'join\s+(\w+)', entry['template'])
        tables = list(set(tables))  # unique tables

        # Gather metadata for each table
        meta = []
        for t in tables:
            meta.append({
                "table": t,
                "row_count": table_size_map.get(t, {}).get('row_count', 'unknown'),
                "size_mb": table_size_map.get(t, {}).get('size_mb', 'unknown'),
                "indexes": index_map.get(t, []),
                "has_partitions": t in partitions_df['TABLE_NAME'].values,
                "constraints": constraints_df[constraints_df['TABLE_NAME'] == t].to_dict('records')
            })

        enriched.append({
            **entry,
            "tables_meta": meta
        })
    return enriched

# Example usage: create enriched query contexts
enriched_queries = enrich_query_context(aggregated_queries)

# Print first enriched context
from pprint import pprint
pprint(enriched_queries[0])


{'avg_exec_time_ms': 190.0,
 'avg_rows_examined': 9750,
 'frequency': 2,
 'index_usage_rate': 0.0,
 'join_types': ['SIMPLE'],
 'tables_meta': [{'constraints': [{'COLUMN_NAME': 'customer_id',
                                   'CONSTRAINT_NAME': 'fk_orders_customer',
                                   'CONSTRAINT_TYPE': 'FOREIGN KEY',
                                   'REFERENCED_COLUMN_NAME': 'id',
                                   'REFERENCED_TABLE_NAME': 'customers',
                                   'TABLE_NAME': 'orders'},
                                  {'COLUMN_NAME': 'id',
                                   'CONSTRAINT_NAME': 'PRIMARY',
                                   'CONSTRAINT_TYPE': 'PRIMARY KEY',
                                   'REFERENCED_COLUMN_NAME': None,
                                   'REFERENCED_TABLE_NAME': None,
                                   'TABLE_NAME': 'orders'}],
                  'has_partitions': False,
                  'indexes': ['id', '

In [8]:
def detect_optimization_candidates(enriched_queries,
                                   slow_time_ms=1000,
                                   high_freq=2,
                                   poor_selectivity_ratio=50,
                                   large_table_mb=500):
    """
    Detects candidate queries for different optimizations:
    - High execution time + high frequency
    - Poor selectivity (rows scanned >> rows returned)
    - Joins on non-indexed foreign keys
    - Aggregations over huge tables
    """
    candidates = {"slow_and_frequent": [], "poor_selectivity": [],
                  "join_no_index": [], "heavy_aggregation": []}

    for q in enriched_queries:
        # --- High execution time + high frequency ---
        if q["avg_exec_time_ms"] > slow_time_ms and q["frequency"] >= high_freq:
            candidates["slow_and_frequent"].append(q)

        # --- Poor selectivity: use avg_rows_examined / rows returned ---
        rows_returned = max(1, q.get("avg_rows_examined", 1) / poor_selectivity_ratio)  # just for ratio calc
        # Here we simulate rows_sent = avg_rows_examined/ratio, but in practice you'd have avg_rows_sent
        # To keep consistent with earlier aggregated_queries, let's fake rows_sent as avg_rows_examined/100
        avg_rows_sent = max(1, q.get("avg_rows_examined", 1) // 100)
        if (q["avg_rows_examined"] / avg_rows_sent) > poor_selectivity_ratio:
            candidates["poor_selectivity"].append(q)

        # --- Joins on non-indexed foreign keys ---
        for meta in q.get("tables_meta", []):
            for c in meta.get("constraints", []):
                if c.get("CONSTRAINT_TYPE") == "FOREIGN KEY":
                    fk_col = c.get("COLUMN_NAME")
                    # If fk_col not indexed
                    if fk_col not in meta.get("indexes", []):
                        candidates["join_no_index"].append(q)
                        break

        # --- Heavy aggregations on huge tables ---
        if any(agg in q.get("template", "").lower() for agg in ["sum", "count", "avg", "max", "min"]):
            if any(meta.get("size_mb", 0) > large_table_mb for meta in q.get("tables_meta", [])):
                candidates["heavy_aggregation"].append(q)

    return candidates

# --- Example usage ---
candidates = detect_optimization_candidates(enriched_queries)

# Print detected candidates
for category, items in candidates.items():
    print(f"\n=== {category.upper()} ===")
    if not items:
        print("No candidates found.")
    for item in items:
        print(f"- Template: {item['template']}")
        print(f"  Avg Exec Time (ms): {item['avg_exec_time_ms']}")
        print(f"  Frequency: {item['frequency']}")
        print(f"  Avg Rows Examined: {item['avg_rows_examined']}")



=== SLOW_AND_FREQUENT ===
No candidates found.

=== POOR_SELECTIVITY ===
- Template: select * from orders where customer_id = ?;
  Avg Exec Time (ms): 190.0
  Frequency: 2
  Avg Rows Examined: 9750
- Template: select c.name, sum(o.amount) from customers c join orders o on c.id=o.customer_id group by c.name;
  Avg Exec Time (ms): 7200.0
  Frequency: 1
  Avg Rows Examined: 500000

=== JOIN_NO_INDEX ===
No candidates found.

=== HEAVY_AGGREGATION ===
- Template: select c.name, sum(o.amount) from customers c join orders o on c.id=o.customer_id group by c.name;
  Avg Exec Time (ms): 7200.0
  Frequency: 1
  Avg Rows Examined: 500000


In [9]:
from pprint import pprint

def build_llm_context(candidates, top_n=3):
    """
    Build formatted context blocks for LLM recommendations.
    """
    contexts = []
    for category, queries in candidates.items():
        for q in queries[:top_n]:  # limit per category
            meta_lines = []
            for meta in q.get("tables_meta", []):
                constraints = [c['CONSTRAINT_TYPE'] for c in meta.get("constraints", [])]
                meta_lines.append(
                    f"Table '{meta['table']}': {meta.get('row_count','?')} rows, "
                    f"{meta.get('size_mb','?')} MB, Indexes={meta.get('indexes',[])}, "
                    f"Constraints={constraints}, Partitioned={meta.get('has_partitions',False)}"
                )

            context_block = f"""
Category: {category}
Query Template:
{q['template']}

Performance Stats:
- Frequency: {q['frequency']}
- Avg Exec Time: {q['avg_exec_time_ms']} ms
- Avg Rows Examined: {q['avg_rows_examined']}

Table Metadata:
{chr(10).join(meta_lines)}
"""
            contexts.append(context_block.strip())
    return contexts

# --- Build contexts ---
contexts = build_llm_context(candidates)

# --- Example: Print first context block ---
pprint(contexts[1])


('Category: poor_selectivity\n'
 'Query Template:\n'
 'select c.name, sum(o.amount) from customers c join orders o on '
 'c.id=o.customer_id group by c.name;\n'
 '\n'
 'Performance Stats:\n'
 '- Frequency: 1\n'
 '- Avg Exec Time: 7200.0 ms\n'
 '- Avg Rows Examined: 500000\n'
 '\n'
 'Table Metadata:\n'
 "Table 'orders': 500000 rows, 820.5 MB, Indexes=['id', 'customer_id'], "
 "Constraints=['FOREIGN KEY', 'PRIMARY KEY'], Partitioned=False\n"
 "Table 'customers': 50000 rows, 45.3 MB, Indexes=['id'], "
 "Constraints=['PRIMARY KEY'], Partitioned=False")


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [5]:

# Simple generate_reply function
def generate_reply(user_message, max_new_tokens=150):
    """
    Generate a response from Mistral 7B Instruct.

    Args:
        user_message (str): Input text from the user
        max_new_tokens (int): Maximum number of tokens to generate

    Returns:
        str: Generated response
    """
    # Prepare prompt for instruct model
    prompt = f"User: {user_message}\nAssistant:"

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate output tokens
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.7
        )

    # Decode and clean up
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove the prompt part from the response
    reply = response[len(prompt):].strip()
    return reply


In [13]:
prompt = f"<s>[INST] You are a database optimization assistant.\n" \
         "Analyze the following query workload and recommend specific optimizations.\n" \
         "Do not repeat the prompt" \
         "Focus on:\n" \
         "1. Index usage or creation.\n" \
         "2. Query rewrite opportunities (avoid SELECT *).\n" \
         "3. Table partitioning or materialized views.\n" \
         "4. Caching or other performance improvements.\n\n" \
         f"Query:\n{contexts[0]}\n" \
         "[/INST]"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=600,
    temperature=0.5,
    top_p=0.9
)

decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded)  # print first chunk to check


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a database optimization assistant.
Analyze the following query workload and recommend specific optimizations.
Do not repeat the promptFocus on:
1. Index usage or creation.
2. Query rewrite opportunities (avoid SELECT *).
3. Table partitioning or materialized views.
4. Caching or other performance improvements.

Query:
Category: poor_selectivity
Query Template:
select * from orders where customer_id = ?;

Performance Stats:
- Frequency: 2
- Avg Exec Time: 190.0 ms
- Avg Rows Examined: 9750

Table Metadata:
Table 'orders': 500000 rows, 820.5 MB, Indexes=['id', 'customer_id'], Constraints=['FOREIGN KEY', 'PRIMARY KEY'], Partitioned=False
 Based on the provided query workload, here are my recommendations for optimizing the performance:

1. Index Usage or Creation:
   Since the query is filtering on the `customer_id` column, which is indexed, it seems like the index is not being used effectively. The high average execution time and rows examined suggest that a covering index could h